In [ ]:
from mastodon import Mastodon, StreamListener
from kafka import KafkaProducer
import json

mastodon = Mastodon(access_token='FuWrfz3vApHoG9Ub6HmCHfPvLgL9qmyPspwZz9Toyis', api_base_url='https://mastodon.social')

producer = KafkaProducer(
    bootstrap_servers='192.168.1.24:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

class TootListener(StreamListener):
    def on_update(self, status):
        toot_data = {
            'content': status['content'],
            'user': status['account']['username'],
            'timestamp': status['created_at'].isoformat()
        }
        producer.send('mastodonstream', toot_data)

listener = TootListener()
mastodon.stream_public(listener)
